In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from sklearn import preprocessing
from scipy.stats import skew, boxcox
from IPython.core.pylabtools import figsize
import xgboost as xgb
%matplotlib inline

from os import path
to_filename = lambda name: path.join("..", "data", "allstate", name +".csv")

import seaborn as sns
sns.set_style("whitegrid")

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv(to_filename("train"), index_col=0)
test = pd.read_csv(to_filename("test"), index_col=0)
print("shape: train {}, test {}".format(train.shape, test.shape))
print(train.head(2))

shape: train (188318, 131), test (125546, 130)
   cat1 cat2 cat3 cat4 cat5 cat6 cat7 cat8 cat9 cat10   ...        cont6  \
id                                                      ...                
1     A    B    A    B    A    A    A    A    B     A   ...     0.718367   
2     A    B    A    A    A    A    A    A    B     B   ...     0.438917   

       cont7    cont8    cont9   cont10    cont11    cont12    cont13  \
id                                                                      
1   0.335060  0.30260  0.67135  0.83510  0.569745  0.594646  0.822493   
2   0.436585  0.60087  0.35127  0.43919  0.338312  0.366307  0.611431   

      cont14     loss  
id                     
1   0.714843  2213.18  
2   0.304496  1283.60  

[2 rows x 131 columns]


In [3]:
response = np.log(train.loss)

mean_resp = np.mean(response)
std_resp = np.std(response)
response = (response - mean_resp) / std_resp


def restore_pred1(y):
    return np.exp(y)

def restore_pred(y):
    return np.exp(y * std_resp + mean_resp)

In [4]:
cat_features = [col for col in train.columns if col.startswith("cat")]
print("Categorical columns:", cat_features)

('Categorical columns:', ['cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9', 'cat10', 'cat11', 'cat12', 'cat13', 'cat14', 'cat15', 'cat16', 'cat17', 'cat18', 'cat19', 'cat20', 'cat21', 'cat22', 'cat23', 'cat24', 'cat25', 'cat26', 'cat27', 'cat28', 'cat29', 'cat30', 'cat31', 'cat32', 'cat33', 'cat34', 'cat35', 'cat36', 'cat37', 'cat38', 'cat39', 'cat40', 'cat41', 'cat42', 'cat43', 'cat44', 'cat45', 'cat46', 'cat47', 'cat48', 'cat49', 'cat50', 'cat51', 'cat52', 'cat53', 'cat54', 'cat55', 'cat56', 'cat57', 'cat58', 'cat59', 'cat60', 'cat61', 'cat62', 'cat63', 'cat64', 'cat65', 'cat66', 'cat67', 'cat68', 'cat69', 'cat70', 'cat71', 'cat72', 'cat73', 'cat74', 'cat75', 'cat76', 'cat77', 'cat78', 'cat79', 'cat80', 'cat81', 'cat82', 'cat83', 'cat84', 'cat85', 'cat86', 'cat87', 'cat88', 'cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'c

In [ ]:
# Categorical features preprocessing
# Method 1: Encoding categorical features into int
for col in cat_features:
    encd = preprocessing.LabelEncoder()
    encd.fit(train[col].value_counts().index.union(test[col].value_counts().index))
    train[col] = encd.transform(train[col])
    test[col] = encd.transform(test[col])

In [5]:
# Method 2: Using cardinal features for categorical features
col = cat_features[0]
test_col = train[col][:10].copy()
for col in cat_features:
    key_map = response.groupby(train[col]).mean().to_dict()
    train[col] = train[col].replace(key_map)
    for k in set(test[col].value_counts().index).difference(key_map.keys()):
        key_map[k] = np.NAN
    test[col] = test[col].replace(key_map)

In [6]:
# preprocess numerical features
num_features = [col for col in train.columns if col.startswith("cont")]
print("Numerical columns:", num_features)

('Numerical columns:', ['cont1', 'cont2', 'cont3', 'cont4', 'cont5', 'cont6', 'cont7', 'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13', 'cont14'])


In [7]:
# Method 1: Standard Scaler
for col in num_features:
    sc = preprocessing.StandardScaler()
    sc.fit(pd.concat([train[[col]], test[[col]]]))
    train[col] = sc.transform(train[[col]])
    test[col] = sc.transform(test[[col]])

In [ ]:
# study the skewness in the numerical features
skewed_feats = pd.concat([train[num_features], test[num_features]]).apply(lambda x: skew(x.dropna()))
print("Skew in numeric features:", skewed_feats)

In [ ]:
# Method 2: Box-Cox transformation when numerical feature skewness > .25
for feat in skewed_feats[skewed_feats > 0.25].index:
    train[feat], lam = boxcox(train[feat] + 1.)
    test[feat], lam = boxcox(test[feat] + 1.)

In [8]:
dtrain = xgb.DMatrix(train.drop("loss", 1), response)
dtest = xgb.DMatrix(test)

In [9]:
params = {'objective':"reg:linear", 'silent': True, 'max_depth': 7, 'min_child_weight': 1,
          'colsample_bytree': .7, "subsample": 1., 'eta': 0.25, 'eval_metric':'mae',# "n_estimators": 20,
          "gamma": 0.25, "lambda": 0.8, "silent": 1}

In [ ]:
cvresult = xgb.cv(params, dtrain, nfold=3, num_boost_round=50)
print(cvresult)

In [ ]:
cvresult[["test-mae-mean", "train-mae-mean"]].plot()

In [ ]:
cvresult[["test-mae-mean", "train-mae-mean"]].plot()

In [10]:
folds = 4

pred_test = 0.
pred_train = 0.
restored_pred_train = 0.
restored_pred_test = 0.

kf = KFold(n_splits=folds)
kf.split(train)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    train_pd_ind = train.index[train_index]
    test_pd_ind = train.index[test_index]
    train_part, test_part = train.ix[train_pd_ind], train.ix[test_pd_ind]
    
    dtrain_part = xgb.DMatrix(train_part.drop("loss", 1), response[train_pd_ind])
    dtest_part = xgb.DMatrix(test_part.drop("loss", 1), response[test_pd_ind])
    params['seed'] = i * 5 + 100
    clf = xgb.train(params, dtrain_part, num_boost_round=100,
                    evals=[(dtrain_part, "train"), (dtest_part, "eval")])
    
    print("best ntree limit", i, clf.best_ntree_limit)
    this_pred_train = clf.predict(dtrain, ntree_limit=clf.best_ntree_limit)
    print("mae for part train",i, mean_absolute_error(
            train_part.loss, restore_pred(clf.predict(dtrain_part, ntree_limit=clf.best_ntree_limit))))
    print("mae for part test",i, mean_absolute_error(
            test_part.loss, restore_pred(clf.predict(dtest_part, ntree_limit=clf.best_ntree_limit))))
    print("mae for all train",i, mean_absolute_error(train.loss, restore_pred(this_pred_train)))
    
    pred_train += this_pred_train
    restored_pred_train += restore_pred(this_pred_train)
    
    this_pred_test = clf.predict(dtest, ntree_limit=clf.best_ntree_limit)
    pred_test += this_pred_test
    restored_pred_test += restore_pred(this_pred_test)
    

[0]	train-mae:0.789285	eval-mae:0.791576
[1]	train-mae:0.706228	eval-mae:0.708734
[2]	train-mae:0.652257	eval-mae:0.655135
[3]	train-mae:0.616333	eval-mae:0.619677
[4]	train-mae:0.592094	eval-mae:0.595774
[5]	train-mae:0.576196	eval-mae:0.580975
[6]	train-mae:0.563295	eval-mae:0.568493
[7]	train-mae:0.553424	eval-mae:0.559536
[8]	train-mae:0.54697	eval-mae:0.55395
[9]	train-mae:0.540475	eval-mae:0.5484
[10]	train-mae:0.535639	eval-mae:0.544054
[11]	train-mae:0.531341	eval-mae:0.540286
[12]	train-mae:0.527952	eval-mae:0.537789
[13]	train-mae:0.525382	eval-mae:0.535539
[14]	train-mae:0.522958	eval-mae:0.533734
[15]	train-mae:0.521046	eval-mae:0.532606
[16]	train-mae:0.518464	eval-mae:0.530644
[17]	train-mae:0.516953	eval-mae:0.529823
[18]	train-mae:0.515256	eval-mae:0.528644
[19]	train-mae:0.51368	eval-mae:0.527678
[20]	train-mae:0.512474	eval-mae:0.527009
[21]	train-mae:0.510897	eval-mae:0.526043
[22]	train-mae:0.509676	eval-mae:0.5254
[23]	train-mae:0.508131	eval-mae:0.524594
[24]	trai

In [11]:
print("mae final restore after", mean_absolute_error(train.loss, restore_pred(pred_train / folds)))
print("mae final restore before", mean_absolute_error(train.loss, restored_pred_train / folds))

('mae final restore after', 1050.1802405758608)
('mae final restore before', 1048.9870091640296)


In [12]:
import datetime
result = pd.DataFrame({"id": test.index, "loss": restored_pred_test / folds})
result.to_csv("result_restored_before{:%Y%m%d%H%-M}.csv".format(datetime.datetime.now()), index=None)

# Using XGBRegressor and important features

In [ ]:
from sklearn.grid_search import GridSearchCV
from xgboost import XGBRegressor

In [ ]:
params_reg = dict(params)
params_reg.pop("eta")
params_reg.pop('eval_metric')
params_reg.pop('lambda')

In [ ]:
reg = XGBRegressor(**params_reg)
reg.fit(train.drop("loss", 1), train.loss)

In [ ]:
train_predprob = reg.score()

In [ ]:
reg_booster = reg.booster()

In [ ]:
figsize(18, 5)
feat_imp = pd.Series(reg.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')

In [ ]:
important_features = list(feat_imp[feat_imp > 4].index)
print("important features:", important_features)

In [ ]:
dtrain_imp = xgb.DMatrix(train[important_features], train.loss)
cvresult = xgb.cv(params, dtrain_imp, nfold=4, num_boost_round=50)
print(cvresult)

In [ ]:
params2 = {'base_score': 0.1, 'colsample_bytree': 0.9,
 'eta': 0.3,
 'eval_metric': 'mae',
 'max_depth': 7,
 'min_child_weight': 3,
 'n_estimators': 10,
 'objective': 'reg:linear',
 'seed': 1,
 'silent': True}
regb = xgb.train(params2, dtrain_imp, num_boost_round=50, evals=[(dtrain_imp, "train")])

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator=reg, 
 param_grid = param_test1, scoring='neg_mean_squared_error',n_jobs=4, iid=False, cv=5)

In [ ]:
gsearch1.fit(train.drop("loss", 1), train.loss)